In [2]:
import json
from typing import List

# Unstructured for document parsing
from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title

# LangChain components
from langchain_core.documents import Document
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import HumanMessage
# from dotenv import load_dotenv

# load_dotenv()

In [3]:
def partition_document(file_path: str):
    """Extract elements from PDF using unstructured"""
    print(f"📄 Partitioning document: {file_path}")
    
    elements = partition_pdf(
        filename=file_path,  # Path to your PDF file
        strategy="hi_res", # Use the most accurate (but slower) processing method of extraction
        infer_table_structure=True, # Keep tables as structured HTML, not jumbled text
        extract_image_block_types=["Image"], # Grab images found in the PDF
        extract_image_block_to_payload=True # Store images as base64 data you can actually use
    )
    
    print(f"✅ Extracted {len(elements)} elements")
    return elements

# Test with your PDF file
file_path = "./docs/attention-is-all-you-need.pdf"  # Change this to your PDF path
elements = partition_document(file_path)

📄 Partitioning document: ./docs/attention-is-all-you-need.pdf


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [ ]:
# len(elements)
elements